In [19]:
%load_ext autoreload
%autoreload 2

# Signal loader

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
from scipy.signal import stft
from Feature_extract import feature_extractors
import torch
import json 
class THU_006or018_basic():
    def __init__(self, args): # 1hz, 10hz, 15hz,IF
        self.data_loader(args.data_dir,args.target)
        self.data_create()
        if not os.path.exists(args.save_dir):
            os.makedirs(args.save_dir)
        self.save_dir = args.save_dir
        # Load data and labels 
    def data_loader(self,data_dir,target):
        
        self.data = np.load(data_dir + target + '_data.npy').astype(np.float32)
        self.labels = np.load(data_dir + target + '_label.npy').astype(np.float32)
        
    def data_create(self):
       #  Define split ratios
        train_ratio = 0.6
        val_ratio = 0.1
        # Calculate test_ratio to ensure ratios sum to 1
        test_ratio = 1 - (train_ratio + val_ratio)

        # Split indices for each label
        train_indices, val_indices, test_indices = [], [], []

        # TODO: Select a fixed number of samples for each label
        selected_len = 10

        for label in np.unique(self.labels):
            label_indices = np.where(self.labels == label)[0][:selected_len]
            # np.random.shuffle(label_indices)
            
            n_train = int(len(label_indices) * train_ratio)
            n_val = int(len(label_indices) * val_ratio)
            # Remaining indices are for testing
            n_test = len(label_indices) - n_train - n_val

            # Append indices for each set
            train_indices.extend(label_indices[:n_train])
            val_indices.extend(label_indices[n_train:n_train + n_val])
            test_indices.extend(label_indices[n_train + n_val:])

        # # Select indices based on the flag
        # if self.flag == 'train':
        #     selected_indices = train_indices
        # elif self.flag == 'val':
        #     selected_indices = val_indices
        # elif self.flag == 'test':
        #     selected_indices = test_indices
        # else:
        #     raise ValueError("Invalid flag. Please choose from 'train', 'val', or 'test'.")

        self.train_data = self.data[train_indices]
        self.train_label = self.labels[train_indices]

        self.val_data = self.data[val_indices]
        self.val_label = self.labels[val_indices]

        self.test_data = self.data[test_indices]
        self.test_label = self.labels[test_indices]

    def data_save(self,save_dir):
        np.save(save_dir + 'train_data.npy', self.train_data)
        np.save(save_dir + 'train_label.npy', self.train_label)
        np.save(save_dir + 'val_data.npy', self.val_data)
        np.save(save_dir + 'val_label.npy', self.val_label)
        np.save(save_dir + 'test_data.npy', self.test_data)
        np.save(save_dir + 'test_label.npy', self.test_label)

    def data_plot_save(self,time_step_per_sample=1):

        datas = [self.train_data, self.val_data, self.test_data]
        labels = [self.train_label, self.val_label, self.test_label]
        
        
        datasets = {
            'train': [self.train_data, self.train_label],
            'val': [self.val_data, self.val_label],
            'test': [self.test_data, self.test_label]
        }

        t = np.linspace(0, time_step_per_sample,datas[0].shape[1])
        signal_instruct = []

        for key,(data,label) in datasets.items():
            for data_index,(sample,label_sample) in tqdm.tqdm(enumerate(zip(data,label)),total=len(data),desc='Plotting data and labels'):
                sample_one = sample[:,0]

                # 时域
                # plt.plot(t,sample_one, label='Signal')
                # # plt.plot(label[i], label='Label')
                # plt.xlabel('Time')
                # plt.ylabel('Value')
                # plt.legend()
                # plt.savefig(self.save_dir + f'THU_006_data_label_{label_sample}' + str(data_index) + '.jpg', dpi=600)
                # plt.close()
                # 频域
                # 时频图
                # 计算短时傅里叶变换
                frequencies, times, Zxx = stft(sample_one, fs=1.0/time_step_per_sample, nperseg=100, noverlap=50)

                # 绘制时频图
                plt.pcolormesh(times, frequencies, np.abs(Zxx), shading='gouraud')
                plt.title('STFT Magnitude')
                plt.ylabel('Frequency [Hz]')
                plt.xlabel('Time [points]')
                file_name=  f'THU_006_{key}_{label_sample}' + str(data_index) + '_stft.jpg'
                plt.savefig(self.save_dir + file_name, dpi=600)
                plt.close()
                res = self.data_feature(sample_one,file_name)
                signal_instruct.append(res)
        with open('signal_instruct.json', 'w') as f:
            json.dump(signal_instruct, f)
        
        return signal_instruct


    def data_feature(self,sample_one,file_name):
        res = {}
        res['image'] = file_name
        res['feature'] = {}
        for key,feature_extractor in feature_extractors.items():
            res['feature'][key] = feature_extractor(torch.tensor(sample_one)).numpy().item()
        return res
    
    def basic_add_template(self):
        pass
            
        





In [2]:
from argparse import Namespace
args = Namespace()
args.data_dir = '/home/user/data/a_bearing/a_006_THU_pro/'
args.target = '10hz'
args.save_dir = '/home/user/LQ/B_Signal/LLava_signal/data/signal_data/demo/save_dir_stft/'

dataset = THU_006or018_basic(args)
dataset.data_plot_save()

Plotting data and labels:   0%|          | 0/24 [00:00<?, ?it/s]

Plotting data and labels: 100%|██████████| 12/12 [00:04<00:00,  2.93it/s]


[{'image': 'THU_006_train_0.00_stft.jpg',
  'feature': {'Mean': -0.0008710779948160052,
   'Std': 0.01727261394262314,
   'Var': 0.0002983432204928249,
   'Entropy': 0.007543822750449181,
   'AbsMean': 0.013559645973145962,
   'Kurtosis': 3.5381526947021484}},
 {'image': 'THU_006_train_0.01_stft.jpg',
  'feature': {'Mean': -0.0011105708545073867,
   'Std': 0.017081551253795624,
   'Var': 0.0002917794045060873,
   'Entropy': 0.009529337286949158,
   'AbsMean': 0.013419398106634617,
   'Kurtosis': 3.380795955657959}},
 {'image': 'THU_006_train_0.02_stft.jpg',
  'feature': {'Mean': -0.0010894376318901777,
   'Std': 0.017080195248126984,
   'Var': 0.0002917330712080002,
   'Entropy': 0.009353507310152054,
   'AbsMean': 0.01339738629758358,
   'Kurtosis': 3.4682130813598633}},
 {'image': 'THU_006_train_0.03_stft.jpg',
  'feature': {'Mean': -0.0011844004038721323,
   'Std': 0.016963893547654152,
   'Var': 0.00028777369880117476,
   'Entropy': 0.010139438323676586,
   'AbsMean': 0.01336137671

# Feature dict

# Technique report

In [11]:
import json

# def describe_features(features):
#     """
#     Generate a description for each feature in a dictionary.
#     """
#     descriptions = [
#         f"The Mean of the signal is {features['Mean']:.4f}.",
#         f"The Standard Deviation (Std) of the signal is {features['Std']:.4f}.",
#         f"The Variance (Var) of the signal is {features['Var']:.4f}.",
#         f"The Entropy of the signal is {features['Entropy']:.4f}.",
#         f"The Absolute Mean (AbsMean) of the signal is {features['AbsMean']:.4f}.",
#         f"The Kurtosis of the signal is {features['Kurtosis']:.4f}."
#     ]
#     return descriptions

def describe_features(features):
    """
    Generate a description for each feature in a dictionary by iterating over key-value pairs.
    """
    descriptions = []
    questions_title = ["What is the", "Describe the", "What does the", "Explain the", "What can you tell me about the"]
    question = []
    for key, value in features.items():
        # Format the key to be more readable, replacing underscores with spaces and capitalizing each word
        readable_key = ' '.join(word.capitalize() for word in key.split('_'))
        descriptions.append(f"The {readable_key} of the signal is {value:.4f}.")
        question.append(f"{questions_title[np.random.randint(0, len(questions_title))]} {readable_key} of the signal?")

    return question,descriptions

def extract_health_status(image_name):
    """
    Extract health status from the image file name based on the first decimal digit.
    """
    # Splitting the filename to extract the relevant part which contains the health status indicator
    health_indicator = image_name.split('_')[3]
    # Extracting the first decimal digit
    first_decimal_digit = int(health_indicator.split('.')[1][0])
    
    # Determining the health status based on the first decimal digit
    health_status = f"healthy_{first_decimal_digit}" if first_decimal_digit == 0 else f"unhealthy{first_decimal_digit}"
    return health_status

def transform_data(input_data,save_dir = 'save_dir_stft/'):
    """
    Transform the input data into the specified format with conversations.
    """
    transformed_data = []

    for id,item in enumerate(input_data):
        health_status = extract_health_status(item['image'])
        # health_status = "healthy" if float(item['image'].split('_')[2]) == 0.00 else "unhealthy"
        question,feature_descriptions = describe_features(item['feature'])
        
        conversation = []


        conversation.append({"from": "human", "value": "<image>\nDescribe a feature of this signal."})
        conversation.append({"from": "gpt", "value": feature_descriptions[0]})

        for desc in feature_descriptions[1:]:
            conversation.append({"from": "human", "value": question})  # what is the 
            conversation.append({"from": "gpt", "value": desc})


        # final
        conversation.append({"from": "human", "value": f"Describe the health status of this signal."})
        conversation.append({"from": "gpt", "value": f"This signal is categorized as {health_status} based on its characteristics."})

        transformed_item = {
            "id": f"{id}",
            "image": f"{save_dir}{item['image']}",
            "conversations": conversation
        }
        transformed_data.append(transformed_item)

    return transformed_data

# 打开并读取json文件
with open('signal_instruct.json', 'r') as f:
    data = json.load(f)


# Transforming the data
transformed_data = transform_data(data)

# Printing out the transformed data
with open('signal_instruct_data.json', 'w') as f:
    json.dump(transformed_data, f)


# print(json.dumps(transformed_data, indent=4))

# # Example input data
# input_data = [
#     {"image": "THU_006_train_0.00_stft.jpg", "feature": {"Mean": -0.0008710779948160052, "Std": 0.01727261394262314, "Var": 0.0002983432204928249, "Entropy": 0.007543822750449181, "AbsMean": 0.013559645973145962, "Kurtosis": 3.5381526947021484}}
# ]

# # Transforming the data
# transformed_data = transform_data(input_data)

# # Printing out the transformed data
# print(json.dumps(transformed_data, indent=4))
